In [1]:
import json
import polars as pl
import re
from thefuzz import fuzz
from datasets import load_dataset
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util

import pandas as pd

##### Load sample sports and political data

In [117]:
#sports_df = pl.read_csv('~/sports-language-in-politics/data/processed/sports_sample.csv')
politics_df = pl.read_csv('~/sports-language-in-politics/data/processed/politics_sample.csv')  # 3203899
random_df = pl.read_csv('~/sports-language-in-politics/data/processed/random_sample_no_sports.csv') 

In [4]:
random_df = pd.read_csv('~/sports-language-in-politics/data/processed/random_sample_no_sports.csv', usecols=["id", "subreddit"]) 

/var/folders/by/rktr_w596p97pmt8_cbknvs80000gn/T/ipykernel_37490/3484189379.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  random_df = pd.read_csv('~/sports-language-in-politics/data/processed/random_sample_no_sports.csv', usecols=["id", "subreddit"])


In [6]:
len(random_df.groupby('subreddit').agg({"id":"count"}).sort_values("id",ascending=False))

28551

##### Extract sports and political comments and clean

In [11]:
#sports_comments = [re.sub(r"[^a-zA-Z0-9]+", ' ', comment).lower() for comment in sports_df['body'].to_list()]
political_comments = [re.sub(r"[^a-zA-Z0-9]+", ' ', comment).lower() for comment in politics_df['body'].to_list()]
random_comments = [re.sub(r"[^a-zA-Z0-9]+", ' ', comment).lower() for comment in random_df['body'].to_list()]

##### Filter short comments

In [4]:
thresh = 20
random_comments_long = []
political_comments_long = []

r_lens = [len(r.split()) for r in random_comments]
p_lens = [len(c.split()) for c in political_comments]

for i in range(len(random_comments)):
    if r_lens[i] >= thresh:
        random_comments_long.append(random_comments[i])
for i in range(len(political_comments)):
    if p_lens[i] >= thresh:
        political_comments_long.append(political_comments[i])

##### Samples from random and poitical comments

In [12]:
thresh = 50000

#wikitext = load_dataset('wikitext', 'wikitext-103-v1')
# sample size
random_comments = random_comments[:thresh]
political_comments = political_comments[:thresh]

##### token_set_ratio tokenizes both input strings, removes duplicate tokens, and calculates the similarity score based on the intersection and union of the token sets

difflib.ratio -> edit distance

token_set_ratio attempts to rule out differences in the strings. Calls ratio on three particular substring sets and returns the max:

- intersection-only and the intersection with remainder of string one
- intersection-only and the intersection with remainder of string two
- intersection with remainder of one and intersection with remainder of two
  
By splitting up the intersection and remainders of the two strings, we're accounting for both how similar and different the two strings are:

##### look into fuzzy match code

In [38]:
wiki = load_dataset('wikitext','wikitext-103-v1', split='train+test')
wiki_text = wiki['text']
wiki_text = [w for w in wiki_text if w != ''][:thresh]
wiki_text = [re.sub(r"[^a-zA-Z0-9]+", ' ', comment).lower() for comment in wiki_text]

##### Metaphors

In [3]:
with open('/users/Ujan/sports-language-in-politics/data/processed/meta_dict.json', 'r') as fp:
    data = json.load(fp)

meta_list = []
for key, values in data.items():
    meta_list.extend(values)

# improve regex
meta_list = [meta.replace("'", '') for meta in meta_list]
meta_list = [re.sub(r"[^a-zA-Z0-9]+", ' ', meta).lower() for meta in meta_list]

In [4]:
meta_list

['cat and mouse game',
 'play cat and mouse',
 'play possum',
 'roll over and play dead',
 'when the cats away',
 'beside the point',
 'come unstrung',
 'drawback',
 'fall short of the mark',
 'fall short of the target',
 'high strung',
 'hit the bulls eye',
 'let slip the dogs of war',
 'off color',
 'room to swing a cat in',
 'shoot your bolt',
 'slings and arrows',
 'upshot',
 'athletes foot',
 'badger',
 'keep it up',
 'bounce back',
 'rubber check',
 'thats the way the ball bounces',
 'your ball game',
 'ball game',
 'ballpark',
 'ballpark figure',
 'baseball size hail',
 'bat an idea around',
 'bat cleanup',
 'batting a thousand',
 'batting average',
 'bench',
 'benched',
 'big league',
 'blank',
 'bleachers',
 'boner',
 'bronx cheer',
 'bush',
 'bush league',
 'call em as you see em',
 'caught napping',
 'caught off base',
 'chalk something up to experience',
 'change of pace',
 'choke',
 'charley horse',
 'close call',
 'clutch hitter',
 'clutch',
 'come off the bench',
 'come 

In [5]:
no_list = [
    'bout','out','close','ace','up','pawn','win','check','check in',
    'bush','card','pass','flat out','drawback','blank','boner',
    'hotdog','iron man','set up','flat out','prize','debut',
    'hit on','set to','discard','dummy','hold in','fan', 'gentlemen',
    'bench', 'debut', 'mainstream', 'coup', 'lightweight', 'win', 'ace',
    'carnival', 'scam', 'keep it up' 
]

meta_list = [m for m in meta_list if m not in no_list]

political comments

In [53]:
count = 0
for comment in political_comments:
    for meta in meta_list:
        if len(re.findall("\\b"+meta+"\\b", comment))>0:
            count += 1

print(count)
# count for each metaphor

457


random comments

In [54]:
count = 0
for comment in random_comments:
    for meta in meta_list:
        if len(re.findall("\\b"+meta+"\\b", comment))>0:
            count += 1

print(count)
# count for each metaphor

430


In [2]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [60]:
with open('/users/Ujan/sports-language-in-politics/data/processed/sim_dict_pol_10k.json', 'r') as fp:
    data = json.load(fp)

In [71]:
device = 'mps'
model.to(device)
model.device

device(type='mps', index=0)

##### data (10000 samples)

In [108]:
data

{'beside the point': [['beside the point', 0],
  ['beside the point', 0],
  ['besides the point', 1],
  ['beside the point', 0],
  ['beside the point', 0],
  ['beside the point', 0],
  ['beside the point', 0],
  ['beside the point', 0],
  ['besides the point', 1],
  ['besides the point', 1],
  ['besides the point', 1],
  ['beside the point', 0],
  ['beside the point', 0],
  ['beside the point', 0],
  ['beside the point', 0],
  ['beside the point', 0],
  ['besides the point', 1],
  ['beside the point', 0]],
 'drawback': [['drawback', 0],
  ['drawbacks', 1],
  ['drawback', 0],
  ['drawback', 0],
  ['drawbacks', 1]],
 'high strung': [['high strung', 0]],
 'off color': [['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['of color', 1],
  ['

In [72]:
embed_dict = {}
meta_count = 0

meta_bar = tqdm(range(len(data)), position=0)
for meta, val in data.items():
    meta_bar.update(1)
    meta_count += 1
    if meta not in meta_list:
        continue
    meta_embedding = model.encode(meta)
    bar = tqdm(range(len(val)), position=0)
    for match in val:
        bar.update(1)
        if match[1] > 0:  # edit distance more than 0
            match_embedding = model.encode(match[0])
            score = util.cos_sim(meta_embedding, match_embedding)
            #if score < 0.8:
                #continue
            if meta not in embed_dict:
                embed_dict[meta] = [(match, score)]
            else:
                embed_dict[meta].append((match, score))

    if meta_count > 10:
        raise

  0%|          | 0/209 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/416 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: No active exception to reraise

##### semantic scores

In [73]:
embed_dict

{'beside the point': [(['besides the point', 1], tensor([[0.8862]])),
  (['besides the point', 1], tensor([[0.8862]])),
  (['besides the point', 1], tensor([[0.8862]])),
  (['besides the point', 1], tensor([[0.8862]])),
  (['besides the point', 1], tensor([[0.8862]]))],
 'off color': [(['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1], tensor([[0.5584]])),
  (['of color', 1],

##### semantic score > 0.8

In [64]:
embed_dict = {}
meta_count = 0

meta_bar = tqdm(range(len(data)), position=0)
for meta, val in data.items():
    meta_bar.update(1)
    if meta not in meta_list:
        continue
    meta_embedding = model.encode(meta)
    bar = tqdm(range(len(val)), position=0)
    for match in val:
        bar.update(1)
        if match[1] > 0:  # edit distance more than 0
            match_embedding = model.encode(match[0])
            score = util.cos_sim(meta_embedding, match_embedding)
            if score < 0.8:
                continue
            if meta not in embed_dict:
                embed_dict[meta] = [(match, score)]
            else:
                embed_dict[meta].append((match, score))

  0%|          | 0/209 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/416 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6992 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [65]:
embed_dict

{'beside the point': [(['besides the point', 1], tensor([[0.8862]])),
  (['besides the point', 1], tensor([[0.8862]])),
  (['besides the point', 1], tensor([[0.8862]])),
  (['besides the point', 1], tensor([[0.8862]])),
  (['besides the point', 1], tensor([[0.8862]]))],
 'badger': [(['badgers', 1], tensor([[0.8071]]))],
 'farm out': [(['farm up', 2], tensor([[0.8052]]))],
 'it aint over till its over': [(['it aint over til its over', 1],
   tensor([[0.9805]]))],
 'judgment call': [(['judgement call', 1], tensor([[0.9524]])),
  (['judgement call', 1], tensor([[0.9524]])),
  (['judgement calls', 2], tensor([[0.9031]]))],
 'major league': [(['minor league', 2], tensor([[0.8540]]))],
 'minor league': [(['major league', 2], tensor([[0.8540]]))]}

In [ ]:
# 5000 samples (1000 * 5), 20 metaphors

# politics (exact_match + semantic_match) 
40+2 + 44+2 + 45+7 + 41+4 + 35+3 = 223

# random (exact_match + semantic_match) 
34+4 + 33+1 + 35+2 + 31+4 + 35+0 = 179

In [ ]:
1,2-gram -> 2
3-gram -> 3
4-gram -> 4

comment length > 100 chars
10000 samples each -> different thresholds

In [26]:
with open('/users/Ujan/sports-language-in-politics/data/processed/sem_dict_42_10000.json', 'r') as fp:
    data = json.load(fp)

In [27]:
data['beside the point'].keys()

dict_keys(['comment_lengths', 'exact_matches', 'semantic_matches'])

In [28]:
len(data['beside the point']['comment_lengths'])

10000

In [30]:
for l in data['beside the point']['semantic_matches']:
    if len(l)>0:
        print(l)

[['beside the police', 3]]
[['believe the point', 3]]
[['aside the point', 2]]


In [34]:
for l in data['play ball']['exact_matches']:
    if len(l)>0:
        print(l)

[['play ball', 0]]
[['play ball', 0]]


In [8]:
with open('/users/Ujan/sports-language-in-politics/data/processed/sem_dict_politics_42_10000.json', 'r') as fp:
    sem_dict = json.load(fp)

In [5]:
no_list = [
    'franchise', 'on board', 'free for all', 'score', 'check', 'game', 'goal', 'pass', 
    'fan', 'tackle', 'debut', 'prize', 'card', 'draw', 'coup', 'to the point', 'out', 'pro',
    'sport', 'course', 'up', 'tip', 'upside', 'set to', 'bench', 'headline', 'set up'
]

In [6]:
# load metaphors
with open('/users/Ujan/sports-language-in-politics/data/processed/meta_dict_full.json', 'r') as fp:
    data = json.load(fp)
    meta_list = []
    for key, values in data.items():
        meta_list.extend(values)

# filter metaphors
print('filtering metaphors')
meta_list = [meta.replace("'", '') for meta in meta_list]
meta_list = [re.sub(r"[^a-zA-Z0-9]+", ' ', meta).lower() for meta in meta_list]
meta_list = [m for m in meta_list if m not in no_list]

filtering metaphors


In [9]:
dup_dict = {}
meta_count = 0
semantic_meta_matches = {}

meta_bar = tqdm(range(len(sem_dict)), position=0)
for meta, match_list in sem_dict.items():
    meta_bar.update(1)
    meta_count += 1
    semantic_meta_matches[meta] = 0

    if len(match_list) == 0 or meta not in meta_list:
        continue

    dup_dict[meta] = []
    meta_embedding = model.encode(meta)
    bar = tqdm(range(len(match_list)), position=1)
    for match in match_list:
        bar.update(1)
        if match not in dup_dict[meta]:
            match_embedding = model.encode(match)
            score = util.cos_sim(meta_embedding, match_embedding)
            if score < args.sem_thresh:
                continue
            semantic_meta_matches[meta] += 1
            #if meta not in embed_dict:
                #embed_dict[meta] = [match]
            #else:
                #embed_dict[meta].append(match)
            dup_dict[meta].append(match)
        else:
            semantic_meta_matches[meta] += 1

    if meta_count > 20:
        raise

  0%|          | 0/1598 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'args' is not defined